In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import time
import os

In [2]:
#set directory
outputDir = ''

#combine 247 Files
two47Files = ['247_bigten.json', '247_bigtwelve.json', '247_sec.json']
main247 = []

for file in two47Files:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        main247.append(player)

#combine rivals
rivalsFiles = ['rivals_bigten.json', 'rivals_bigtwelve.json', 'rivals_sec.json']
mainRivals = []

for file in rivalsFiles:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        mainRivals.append(player)

#combine cfb ref
cfbRefFiles = ['cfb_ref_bigten.json', 'cfb_ref_bigtwelve.json', 'cfb_ref_sec.json']
mainCfbRef = []

for file in cfbRefFiles:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        mainCfbRef.append(player)
                     

#Get NFL
nfl_draft_data = json.load(open('nfldraft.json'))

#All American
aaData = json.load(open('aaSelections.json'))

#combine All Conference
allConfFiles = ['SECAllConf.json', 'Ten12AllConf.json']
mainAllConf = []

for file in allConfFiles:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        mainAllConf.append(player)

#Test Collections
two47_test = [{'school': 'maryland', 'year': '2002', '247_player': 'Randy Earle', '247_hs': 'Farmingdale Senior', '247_city': 'Farmingdale', '247_state': '1', '247_ranking': '0.9000', '247_pos': '9', '247_height': '75.0', '247_weight': '225', '247_stars': '4', '247_natl': '140'},{'school': 'maryland', 'year': '2002', '247_player': 'Andrew Thornton', '247_hs': 'Farmingdale Senior', '247_city': 'Farmingdale', '247_state': '1', '247_ranking': '0.9000', '247_pos': '9', '247_height': '75.0', '247_weight': '225', '247_stars': '4', '247_natl': '140'}]
nfl_test = [{'year': '2005', 'draft_round': '1', 'draft_pick': '12', 'team': 'SDG', 'player': 'Shawne Merriman', 'pos': 'LB', 'all_pros_first_team': '1', 'pro_bowls': '3', 'years_as_primary_starter': '4', 'g': '75', 'college_id': 'Maryland'}, {'year': '2005', 'draft_round': '1', 'draft_pick': '12', 'team': 'SDG', 'player': 'Joseph Holm', 'pos': 'LB', 'all_pros_first_team': '1', 'pro_bowls': '3', 'years_as_primary_starter': '4', 'g': '75', 'college_id': 'Maryland'}]
#rivals_test= [{'school': 'maryland', 'year': '2002', 'rivals_player': 'Jo Jo Walker', 'rivals_city': 'Carrollton', 'rivals_state': 'TX', 'rivals_hs': 'Creekview', 'rivals_pos': 'DB', 'rivals_height': 70.0, 'rivals_weight': 165.0, 'rivals_stars': 3, 'rivals_natlrk': None, 'rivals_posrk': 38, 'rivals_staterk': None}]
#Declare Field Configs
idConfigs = {'rivals' : ['playerName', 'school'], '247': ['playerName', 'school'], 'nfl' : ['player', 'college_id'], 'aa' : ['player', 'school'], 'ac' : ['playerName', 'school'], 'cr' : ['playerName', 'school']}
fieldAgg = '_'
sourceSchema = {'nfl': ['year', 'draft_round', 'draft_pick', 'team', 'pos','all_pros_first_team','pro_bowls','years_as_primary_starter','g'], 'aa' : ['year', 'afca', 'ap', 'fwaa', 'tsn', 'wcff'], 'ac' : ['year', 'coaches', 'media'], 'rivals' : ['playerName', 'city', 'state', 'highSchool', 'position', 'height', 'weight', 'stars', 'nationalRank', 'positionRank', 'stateRank'], '247' : ['playerName', 'highSchool', 'city', 'state', 'position', 'height', 'weight','nationalRank', 'positionRank', 'stateRank'], 'cr' : ['playerName', 'position', 'school', 'gamesPlayed']}

In [3]:
##the issue here is that i['ID'] is a list, not a string, even though you are trying to fix that

In [79]:
##ID consists of two components
##Player Name - which is coming from a string
##School - which can be either a list or a sting.  And the list can have either 1 or 2 parts
##Final ID for everyone - list, potentially multivalued where each part equals name+_+school

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= []
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                for x in i[val]:
                    multiFinalID = []
                    multiFinalID.append(x.lower().replace(" ", "-").replace("st.", "state"))
                    #print(multiFinalID)
                i[val] = multiFinalID
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
                #i['ID'] = multiFinalID
            elif (type(val) is not list):
                i[val] = i[val].lower().replace(" ", "-").replace("st.", "state")
                #print(fieldList[idx])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID += i[val] + fieldAgg
                i['ID'] = finalID
        i['ID'] = finalID
        finalID=''

#TEST CASES
#for i in two47_test:
#    createNewID(idConfigs['247'], two47_test, fieldAgg)
#print(two47_test)


In [80]:
def comparator (field1, field2):
    areTheyEqual = False
    if field2 is not list:
        if equals(field1, field2):
            areTheyEqual =True
    elif field2 is list:
        for x in field2:
            if (field1 == x):
                areTheyEqual = True
    return areTheyEqual

In [87]:
#function to join to nfl draft
def joinData (masterDict, secondDict, schema):
    for player in masterDict:
        for record in secondDict:
            if (comparator(player['ID'],record['ID'])):
                player[schema] = {}
                for x in sourceSchema[schema]:
                    try:
                        player[schema][x] = record[x]
                    except KeyError:
                        print(player['ID'])
                break;


In [88]:
def equals(first, second):
    # Return true if the two ints are equal.
    return first == second

In [89]:
createNewID(idConfigs['247'], main247, fieldAgg)
createNewID(idConfigs['rivals'], mainRivals, fieldAgg)
createNewID(idConfigs['nfl'], nfl_draft_data, fieldAgg)
createNewID(idConfigs['aa'], aaData, fieldAgg)
createNewID(idConfigs['ac'], mainAllConf, fieldAgg)

In [90]:
#print(mainCfbRef)
createNewID(idConfigs['cr'], mainCfbRef, fieldAgg)
print(mainCfbRef)

[{'playerName': 'aaron-bailey', 'school': ['illinois'], 'position': 'QB', 'gamesPlayed': 13, 'ID': ['a', 'a', 'r', 'o', 'n', '-', 'b', 'a', 'i', 'l', 'e', 'y', '_', 'i', 'l', 'l', 'i', 'n', 'o', 'i', 's']}, {'playerName': 'aaron-gress', 'school': ['illinois'], 'position': 'LB', 'gamesPlayed': 25, 'ID': 'aaron-gress_illinois'}, {'playerName': 'abe-cajuste', 'school': ['illinois'], 'position': 'DL', 'gamesPlayed': 1, 'ID': 'abe-cajuste_illinois'}, {'playerName': 'adam-solomon', 'school': ['illinois'], 'position': 'OL', 'ID': 'adam-solomon_illinois'}, {'playerName': 'ade-adeyemo', 'school': ['illinois'], 'position': 'WR', 'gamesPlayed': 12, 'ID': 'ade-adeyemo_illinois'}, {'playerName': 'ahmari-hayes', 'school': ['illinois'], 'position': 'DB', 'gamesPlayed': 7, 'ID': 'ahmari-hayes_illinois'}, {'playerName': 'ainslie-johnson', 'school': ['illinois'], 'position': 'TE', 'gamesPlayed': 5, 'ID': 'ainslie-johnson_illinois'}, {'playerName': 'aj-bush', 'school': ['virginiatech'], 'position': 'QB',

In [91]:
# This creates the root of each recruit in the JSON.  Anything that can be pulled out from the various sources is here
masterData = []
for recruit in main247:
    masterRecruit = {}
    masterRecruit['ID'] = recruit['ID']
    masterRecruit['year'] = recruit['year']
    masterRecruit['school'] = recruit['school']
    masterRecruit['compositeScore'] = recruit['compRating']
    masterRecruit['compositeStars'] = recruit['compStars']
    masterData.append(masterRecruit)

In [92]:
joinData(masterData, main247, '247')
joinData(masterData,mainRivals, 'rivals')
joinData(masterData,nfl_draft_data,  'nfl')
joinData(masterData, aaData,  'aa')
joinData(masterData, mainAllConf, 'ac')
joinData(masterData, mainCfbRef, 'cr')


remond-willis_illinois
pat-flavin_illinois
chris-boles_illinois
chris-o'connor_illinois
luke-nelson_illinois
joe-spencer_illinois
austin-schmidt_illinois
tyrin-stone-davis_illinois
gabe-megginson_illinois
adam-solomon_illinois
jimmy-fitzgerald_illinois
zeke-martin_illinois
zach-heath_illinois
wes-rogers_indiana
dimitric-camiel_indiana
dan-feeney_indiana
aaron-del-grosso_indiana
mike-barwick-jr._indiana
deandre-herron_indiana
timothy-gardner_indiana
austin-king_indiana
hunter-littlejohn_indiana
joe-belden_indiana
davondre-love_indiana
jack-trainor_indiana
chris-gajcak_indiana
mackenzie-nworah_indiana
britt-beery_indiana
jack-cardillo_indiana
aidan-rafferty_indiana
jonathan-king_indiana
nick-marozas_indiana
jacolby-hewitt_indiana
jordan-jusevitch_indiana
chase-wyatt_indiana
beau-robbins_indiana
da'shaun-brown_indiana
matthew-bedford_indiana
larry-tracy-iii_indiana
jeramy-passmore_indiana
antoine-whitner-jr._indiana
jordan-jakes_indiana
tim-weaver_indiana
sean-wracher_indiana
damarjhe-lew

gabe-carimi_wisconsin
kevin-zeitler_wisconsin
ray-ball_wisconsin
tyler-marz_wisconsin
dan-voltz_wisconsin
walker-williams_wisconsin
ryan-ramczyk_wisconsin
george-panos_wisconsin
jeremy-patterson_wisconsin
micah-kapoi_wisconsin
kevin-estes_wisconsin
david-moorman_wisconsin
nick-thomas_wisconsin
andrew-james_wisconsin
luke-benzschawel_wisconsin
pat-colbert_baylor
mo-porter_baylor
sean-muir_baylor
josh-pelzel_baylor
josh-malin_baylor
patrick-lawrence_baylor
mo-porter_baylor
jordan-tolbert_baylor
riley-daniel_baylor
xavier-newman_baylor
khalil-keith_baylor
jason-moore_baylor
ryan-miller_baylor
connor-galvin_baylor
jackson-kimble_baylor
casey-phillips_baylor
jackson-gleeson_baylor
jaylen-ellis_baylor
tyrone-brown_baylor
garmon-randolph_baylor
logan-compton_baylor
hakeem-vance_baylor
sam-snyder_baylor
yusuf-terry_baylor
brandon-white_baylor
harrison-white_baylor
solomon-turner_baylor
blake-bedier_baylor
elijah-ellis_baylor
davis-divall_baylor
james-sylvester_baylor
taye-mcwilliams_baylor
lor

luke-fortner_kentucky
naasir-watkins_kentucky
austin-dotson_kentucky
quintin-wilson_kentucky
nick-lewis_kentucky
kenneth-horsey_kentucky
isaiah-gibson_kentucky
nik-scalzo_kentucky
eli-cox_kentucky
jake-pope_kentucky
joel-williams_kentucky
john-young_kentucky
jutahn-mcclain_kentucky
deondre-buford_kentucky
beau-allen_kentucky
octavious-oxendine_kentucky
kalil-branham_kentucky
andru-phillips_kentucky
d'eryk-jackson_kentucky
vadal-alexander_lsu
ethan-pocic_lsu
josh-boutte_lsu
andy-dodd_lsu
kevin-spears_lsu
k.j.-malone_lsu
clifton-garrett_lsu
tony-upchurch_lsu
brandon-bergeron_lsu
maea-teuhema_lsu
toby-weathersby_lsu
chidi-okeke_lsu
adrian-magee_lsu
hanner-shipley_lsu
austin-deculus_lsu
ed-ingram_lsu
jarell-cherry_lsu
chasen-hines_lsu
dare-rosenthal_lsu
cameron-wire_lsu
kardell-thomas_lsu
ray-parker_lsu
anthony-bradford_lsu
raydarious-jones_lsu
thomas-perry_lsu
charles-turner_lsu
soni-fonua_lsu
desmond-little_lsu
quentin-skinner_lsu
arik-gilbert_lsu
eli-ricks_lsu
kayshon-boutte_lsu
jaqueli

In [94]:
with open("final_output.json", "w") as write_file:
    json.dump(masterData, write_file)

In [ ]:
## NOTE: Bag of Words depends on strings being converted to floats for comparison.  So none of this works.  Pivoting to the levensthein Distance

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

X = [['stefondiggs_maryland', 'stefondiggs_maryland'],['kjhamler_pennstate', 'kjhamler_pennst.'],['trevondiggs_alabama', 'trevondiggs_alabama'],['alexsmith_utah', 'alexsmith_texas'],['kellenmonde_texasam', 'kellenmond_texasa&m']]
Y = ['true','true','true','false','true']

P = [['djmoore_maryland', 'djmoore_maryland'], ['colinkapernick_utahstate', 'collinkapernick_utahst']]

c = DecisionTreeClassifier()
c = c.fit(X,Y)
print("Prediction: " + str(c.predict(P)))

knn = KNeighborsClassifier()
knn.fit(X,Y)
print ("2 Using K Neighbors Classifier Prediction is " + str(knn.predict(P)))

mlpc = MLPClassifier()
mlpc.fit(X,Y)
print ("3) Using MLPC Classifier Prediction is " + str(mlpc.predict(P)))

rfor = RandomForestClassifier()
rfor.fit(X,Y)
print ("4) Using RandomForestClassifier Prediction is " + str(rfor.predict(P)))

In [104]:
print(json.loads(open("output_files\\247_bigten.json", "r", encoding="utf-8").read()))

[{'school': 'illinois', 'year': '2002', 'playerName': 'James McGill', 'highSchool': 'Bakersfield College', 'city': 'Bakersfield', 'state': 'CA', 'compRating': '0.9333', 'position': 'S', 'height': 74.0, 'weight': '215', 'compStars': 4, 'nationalRank': '22', 'positionRank': '2', 'stateRank': '9'}, {'school': 'illinois', 'year': '2002', 'playerName': 'Ibrahim Halsey', 'highSchool': 'Elizabeth', 'city': 'Elizabeth', 'state': 'NJ', 'compRating': '0.8667', 'position': 'RB', 'height': 71.0, 'weight': '190', 'compStars': 3, 'nationalRank': '306', 'positionRank': '24', 'stateRank': '6'}, {'school': 'illinois', 'year': '2002', 'playerName': 'Cyruss Garrett', 'highSchool': 'Romeoville', 'city': 'Romeoville', 'state': 'IL', 'compRating': '0.8667', 'position': 'OLB', 'height': 75.0, 'weight': '220', 'compStars': 3, 'nationalRank': '348', 'positionRank': '19', 'stateRank': '6'}, {'school': 'illinois', 'year': '2002', 'playerName': 'Mel Bryant', 'highSchool': 'Muncie Central', 'city': 'Muncie', 'stat